## Basic Statistics on UKB dataset

Set up global variables and import modules

In [1]:
import os
import sys
import pandas as pd
from matplotlib import pyplot as plt

ukb_root = '/project_freenas/3022017.02/UKB'
sys.path.append(os.path.join(ukb_root,'scripts'))
from ukb_utils import get_variables_UKB, lookup_UKB

ukb_idp_dir = os.path.join(ukb_root,'phenotypes','current')

Read dataframe

In [21]:
basic_demo = pd.read_csv(os.path.join(ukb_idp_dir,'01_basic_demographics.csv'),nrows=1)
brain_IDPs = pd.read_csv(os.path.join(ukb_idp_dir,'31_brain_IDPs.csv'),nrows=1)

Lookup field IDs with Lookup_UKB function <br>
This will help figure out what fields are present in the dataframe

In [ ]:
def list_field_names(data_frame):
  fields_ID = data_frame.columns
  field_list = pd.DataFrame(columns=['Field', 'FieldID'])
  for idf in fields_ID[1:]:
    fld = lookup_UKB(field_ids=[idf]) 
    field_list = field_list.append(fld)
  return field_list

list_field_names(brain_IDPs)